In [1]:
import pandas as pd
rna_data = pd.read_csv("gene_data/rna_common_complete.csv")
rna_data = rna_data.sort_values(by=['sn','period']).reset_index(drop=True)

In [2]:
X_og_shape = rna_data.drop(['sn','group','caarms_status','period'],axis=1).values
X_reshaped = X_og_shape.reshape(len(set(rna_data['sn'])), 3, X_og_shape.shape[1])
labels_group = rna_data[rna_data['period'] == 24]['group'].values
labels = [0 if i == 'C' else 1 for i in labels_group]

In [3]:
import torch
from neucube.utils import SNR
from neucube.utils import interpolate
from neucube.encoder import Delta

ratios = SNR(X_reshaped[:,0,:], labels)
top_idx = torch.argsort(ratios, descending=True)[0:20]
X_reshaped_topidx = X_reshaped[:,:,top_idx]
interpolated_X = interpolate(X_reshaped_topidx, num_points=104)

encoder = Delta(threshold=0.008)
X = encoder.encode_dataset(interpolated_X)
y = torch.tensor(labels)

In [7]:
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex
def objective_function(res_, X_stimuli, labels, params):
    a, b, c, d = params
    res_.update_parms(a=a, b=b, c=c, d=d)
    out_spikes = res_.simulate(X_stimuli, mem_thr=30, train=False, verbose=False)
    sampler = TemporalBinning(bin_size=10)
    state_vec = sampler.sample(out_spikes)
    return SeparationIndex(state_vec, labels).item()

In [5]:
from neucube import IzhReservoir
izh_res = IzhReservoir(inputs=X.shape[2], c=0.7, l=0.18, input_conn_prob=0.85)
izh_res.set_exc_parms(a=0.06, b=0.55, c=-55, d=3)
izh_res.set_inh_parms(a=0.01, b=0.2, c=-65, d=8)

In [8]:
import nevergrad as ng
import numpy as np
from functools import partial

parametrization = ng.p.Tuple(
    ng.p.Array(init=izh_res.a.cpu()),
    ng.p.Array(init=izh_res.b.cpu()), 
    ng.p.Array(init=izh_res.c.cpu()), 
    ng.p.Array(init=izh_res.d.cpu())  
)

# Ensure that the arrays can take on values in the specified range
parametrization[0].set_bounds(lower=0.01, upper=0.2)
parametrization[1].set_bounds(lower=0.2, upper=0.55)
parametrization[2].set_bounds(lower=-65, upper=-45)
parametrization[3].set_bounds(lower=2, upper=8)

optimizer = ng.optimizers.OnePlusOne(parametrization=parametrization, budget=10)
partial_objective_function = partial(objective_function, res_=izh_res, X_stimuli=X, labels=y)

def print_progress(optimizer, n_iterations):
    for i in range(n_iterations):
        x = optimizer.ask()
        loss = -partial_objective_function(params=x.value)
        optimizer.tell(x, loss)
        if (i + 1) % 2 == 0:
            print(f"Iteration {i + 1}/{n_iterations}, Current loss: {loss}")

n_iterations = 10
print_progress(optimizer, n_iterations)
recommendation = optimizer.provide_recommendation()
optimal_a, optimal_b, optimal_c, optimal_d = recommendation.value

print("Optimal x1:", optimal_a[0:10])
print("Optimal x2:", optimal_b[0:10])
print("Optimal x3:", optimal_c[0:10])
print("Optimal x4:", optimal_d[0:10])

c:\Users\singh\miniconda3\Lib\site-packages\nevergrad\parametrization\_datalayers.py:107: NevergradRuntimeWarning: Bounds are 0.19 sigma away from each other at the closest, you should aim for at least 3 for better quality.
  warnings.warn(
c:\Users\singh\miniconda3\Lib\site-packages\nevergrad\parametrization\_datalayers.py:107: NevergradRuntimeWarning: Bounds are 0.35000000000000003 sigma away from each other at the closest, you should aim for at least 3 for better quality.
  warnings.warn(
100%|██████████| 115/115 [00:26<00:00,  4.35it/s]


Iteration 2/10, Current loss: -0.011987916193902493


100%|██████████| 115/115 [00:23<00:00,  4.87it/s]


Iteration 4/10, Current loss: -0.01181294396519661


100%|██████████| 115/115 [00:23<00:00,  4.84it/s]


Iteration 6/10, Current loss: -0.01178249716758728


100%|██████████| 115/115 [00:23<00:00,  4.84it/s]


Iteration 8/10, Current loss: -0.011899088509380817


100%|██████████| 115/115 [00:24<00:00,  4.78it/s]

Iteration 10/10, Current loss: -0.011919310316443443
Optimal x1: [0.01       0.2        0.01       0.2        0.2        0.2
 0.2        0.2        0.2        0.2        0.2        0.2
 0.01       0.2        0.2        0.14090163 0.01       0.2
 0.2        0.10844268 0.01       0.01       0.2        0.01
 0.2        0.01       0.2        0.2        0.2        0.2
 0.2        0.01       0.2        0.01       0.01       0.14108771
 0.01       0.01       0.01       0.01       0.01       0.01
 0.2        0.01       0.2        0.18936654 0.01       0.01
 0.2        0.2        0.2        0.01       0.2        0.01
 0.2        0.06741658 0.2        0.2        0.01       0.2
 0.01       0.01       0.2        0.2        0.2        0.16130637
 0.11510072 0.01       0.01       0.01       0.2        0.2
 0.01389667 0.01       0.2        0.01727851 0.10994964 0.2
 0.01       0.2        0.01       0.01       0.01       0.2
 0.01       0.01       0.01903529 0.2        0.01       0.01
 0.2        0.04

In [ ]:
izh_res.update_parms(a=optimal_a, b=optimal_b, c=optimal_c, d=optimal_d)
opt_spike = izh_res.simulate(X, mem_thr=30, train=False, verbose=True)

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex

sampler = TemporalBinning(bin_size=10)
opt_state_vec = sampler.sample(opt_spike)
print(opt_state_vec.shape)

num_folds = 10
kf = KFold(n_splits=num_folds)

true_labels = []
predicted_labels = []

for train_index, test_index in tqdm(kf.split(opt_state_vec)):
    X_train_fold, X_test_fold = opt_state_vec[train_index], opt_state_vec[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    svm = SVC(kernel='linear', C=2)  # You can specify different kernels ('linear', 'poly', 'rbf', etc.)
    svm.fit(X_train_fold, y_train_fold)
    y_pred = svm.predict(X_test_fold)
    true_labels.extend(y_test_fold)
    predicted_labels.extend(y_pred)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("10-Fold Cross-Validation Accuracy:", accuracy)
print(confusion_matrix(true_labels, predicted_labels))
print("separation:", SeparationIndex(opt_state_vec, y))